In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
from tqdm import tqdm
import numpy as np

In [4]:
df = pd.read_csv('price_list_final.csv')

In [5]:
df.head()

,Unnamed: 0,Date,Name,Year,Month,Gu,Dong,Address,Building Year,Floor,...,Middle-Aged,Old Age,Elementary Schools Within 700m,Nearby Hangang,Prestigious School Within 2500m,Market Within 1km,Hospital Within 1km,Park Presence Within 1km,Jeonse Index,Construction Firm in Rank
0,0,20231231,블루힐하우스,2023,202312,서초구,잠원동,서초구 잠원동 0071-0001,24.0,2,...,0.420339,0.129670,1,1,2,10,1,1,53.7,0
1,1,20231231,신내6대주,2023,202312,중랑구,신내동,중랑구 신내동 0650-0000,27.0,6,...,0.405969,0.196365,3,0,0,2,2,1,53.7,0
2,2,20231231,삼익대청아파트,2023,202312,강남구,개포동,강남구 개포동 0012-0000,31.0,8,...,0.399930,0.162157,4,0,7,4,0,1,53.7,0
3,3,20231231,래미안에스티움,2023,202312,영등포구,신길동,영등포구 신길동 4950-0000,6.0,14,...,0.380671,0.181975,4,0,0,2,3,1,53.7,2
4,4,20231231,동부(105동~106동),2023,202312,구로구,오류동,구로구 오류동 0339-0000,24.0,10,...,0.366548,0.195126,1,0,2,10,16,1,53.7,0


In [6]:
df['Gu_encoding'] = df['Gu'].astype('category').cat.codes + 1
df['Dong_encoding'] = df.groupby('Gu').cumcount() + 1
df['Gu_encoding'] = df['Gu_encoding'].astype('int64')
df['Dong'] = df['Gu_encoding'] * 100 + df['Dong_encoding']

#동을 구에 맞게 인코딩합니다.

In [7]:
X = df[['Dong', 'Building Year', 'Floor', 'Actual Price Index', 'Living Price Index','Interest Rate','Distance to MC','Distance to NS','Bus Station Within 500m', 'Total Population', 'Children', 'Adolescent',	'Youth','Middle-Aged','Old Age', 'Elementary Schools Within 700m','Nearby Hangang', 'Prestigious School Within 2500m', 'Market Within 1km',	'Hospital Within 1km',	'Park Presence Within 1km', 'Construction Firm in Rank', 'Jeonse Index']]
y = df['Price per Area Log Scale']

In [8]:
X_train_test, X_val_test, y_train_test, y_val_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 그 다음, 훈련 세트를 다시 훈련 세트와 검증 세트로 분할
X_train, X_val, y_train, y_val = train_test_split(X_train_test, y_train_test, test_size=0.25, random_state=42)

In [9]:
rf = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)

rf.fit(X_train, y_train)

predictions = rf.predict(X_val)

# 성능 지표 계산
r2 = r2_score(y_val, predictions)
mse = mean_squared_error(y_val, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_val, predictions)

# 성능 지표 출력
print(f'R^2: {r2:.4f}')
print(f'MSE: {mse:.4f}')
print(f'RMSE: {rmse:.4f}')
print(f'MAE: {mae:.4f}')



R^2: 0.8865
MSE: 0.0240
RMSE: 0.1548
MAE: 0.1079
